# 读取 final_solutions 文件，拟合 R-NSGA-2 参考点
**独立代码块，需要自定义chkpt_dir**
 - 加载或定义现有的 NSGA-2 最终前沿解（3D 散点）。
 - 分段定义/拟合得到三维坐标网格（作为 R-NSGA-2 的参考点），并绘制为蓝色散点。
 - 在 3D 图中，同步绘制若干半透明球体(red spheres) 来表示每个参考点周围的 ϵ 邻域。

In [ ]:
%matplotlib widget

import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
# 指定一个支持中文的字体，比如 Windows 上常见的 "SimHei" (黑体)
matplotlib.rcParams["font.sans-serif"] = ["SimHei"]  
matplotlib.rcParams["axes.unicode_minus"] = False    # 正常显示负号

chkpt_dir = r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\0210早-nsga2-180群150代-收敛"

# ---------- 读取 NSGA-2 的非支配解 ----------
filename_nsga2 = os.path.join(chkpt_dir, "final_solutions.npz")
data_nsga2 = np.load(filename_nsga2)
F_nsga2 = data_nsga2["F"]  # (n_nd1, n_obj)

# 对应于 MATLAB 代码: mat_data = loadmat("ParetoSol_nsga2.mat")

ParetoSol_nsga2 = np.zeros((len(F_nsga2[:,0]), 3))
ParetoSol_nsga2[:,0] = F_nsga2[:,0]
ParetoSol_nsga2[:,1] = F_nsga2[:,1]
ParetoSol_nsga2[:,2] = F_nsga2[:,2]

##################################################
# 根据分段函数，生成 (Y, Z)，以及指定若干 X_range 形成网格点
##################################################

# (A) 第一段 [Y_min_1, Y_max_1], 步长= step_1
Y_min_1, Y_max_1 = 100, 110
step_1 = 20
Y_vals_1 = np.arange(Y_min_1, Y_max_1+1, step_1)
Z_vals_1 = -3.9146 * Y_vals_1 + 751.7518

# (B) 第二段 给定若干离散点
Y_vals_2 = np.array([150, 250, 400])
Z_vals_2 = 225.4385 * np.exp((-0.0028) * Y_vals_2) + \
           172.2804 * np.exp((0.0004) * Y_vals_2) - 10

# (C) 第三段 [500, 900], 步长=150
Y_vals_3 = np.arange(500, 900+1, 150)
Z_vals_3 = -0.0232 * Y_vals_3 + 276.4945

# 合并三段
Y_all = np.concatenate([Y_vals_1, Y_vals_2, Y_vals_3])
Z_all = np.concatenate([Z_vals_1, Z_vals_2, Z_vals_3])

# 再给 X 指定若干离散点:
X_range = np.array([-250, -215, -165, -120, -80])

# 构建网格 (X_grid, YZ_index)
X_grid, idx_grid = np.meshgrid(X_range, np.arange(len(Y_all)))
X_grid = X_grid.ravel()      # 拉成一维
idx_grid = idx_grid.ravel()  # 索引

Y_grid = Y_all[idx_grid]
Z_grid = Z_all[idx_grid]

# 删除条件: X >= -120 AND Y > 350
cond_delete = (X_grid >= -120) & (Y_grid > 350)
X_grid = X_grid[~cond_delete]
Y_grid = Y_grid[~cond_delete]
Z_grid = Z_grid[~cond_delete]

# 参考点(3D) 坐标集
XYZ_fitted = np.vstack([X_grid, Y_grid, Z_grid]).T  # shape (M, 3)

# 保存点集
np.save('RefPnt_fromNSGA2.npy',XYZ_fitted)

##################################################
# 三维可视化
#    - NSGA-2 前沿解 (红点)
#    - 拟合曲面散点 (蓝点)
#    - 为每个参考点画一个半透明球（半径 epsilon）
##################################################

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

# (A) 绘制蓝色散点 (R-NSGA-2 的参考点)
ax.scatter(X_grid, Y_grid, Z_grid, c='b', marker='o', s=20, label='Fitted Surface Points')

# (B) 绘制 NSGA-2 最终前沿解 (红点)
ax.scatter(ParetoSol_nsga2[:,0],
           ParetoSol_nsga2[:,1],
           ParetoSol_nsga2[:,2],
           c='r', marker='o', s=20, label='NSGA-2 Pareto Set')

# 设置坐标轴名称
ax.set_xlabel('X轴')
ax.set_ylabel('Y轴')
ax.set_zlabel('Z轴')

# 试验一个 epsilon (球半径)
epsilon = 15.0

# (C) 在每个参考点处，绘制一个半透明球
#     球面参数化: x = r*sin(phi)*cos(theta), y = r*sin(phi)*sin(theta), z = r*cos(phi)
phi = np.linspace(0, np.pi, 8)      # 球体表面网格的分辨率
theta = np.linspace(0, 2*np.pi, 8)
phi, theta = np.meshgrid(phi, theta)

for i in range(XYZ_fitted.shape[0]):
    x0, y0, z0 = XYZ_fitted[i,:]
    
    # 生成球面 (在原点半径 epsilon)
    xs = epsilon * np.sin(phi) * np.cos(theta)
    ys = epsilon * np.sin(phi) * np.sin(theta)
    zs = epsilon * np.cos(phi)
    
    # 平移到 (x0, y0, z0)
    xs += x0
    ys += y0
    zs += z0
    
    # 绘制球面
    #  - alpha=0.2 设置半透明
    #  - color='red' 或者 'r'
    ax.plot_surface(xs, ys, zs, color='r', alpha=0.15, linewidth=0)

plt.title('3D Compare: Fitted Surface (blue) & NSGA-2 Frontier (red) & Spheres')
ax.legend()
ax.view_init(elev=25, azim=-60)  # 视角可调
plt.show()
